<a href="https://colab.research.google.com/github/siddh1990/Data-Driven-Pump-it-Up/blob/main/Data_Driven_Pump_it_Up_Raw_Catboost_to_team_17_12_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_x=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Train set values.csv")
data_y=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Train set labels.csv")

In [ ]:
data=pd.merge(data_x,data_y,on="id",how="inner")

## Splitting the data into training and testing

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,train_size=0.7,random_state=42,stratify=data_y["status_group"])

## Data Exploration

In [ ]:
#Numerical Features Description
x_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,41580.000000,41580.000000,41580.000000,41580.000000,4.158000e+04,41580.000000,41580.000000,41580.000000,41580.000000,41580.000000
mean,37097.435065,327.861776,668.457119,34.115174,-5.712070e+00,0.513949,15.352549,5.643747,178.159307,1305.088480
std,21449.071188,3338.635977,692.792692,6.498510,2.939523e+00,13.574083,17.695093,9.671902,447.796478,950.207371
min,1.000000,0.000000,-63.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18561.750000,0.000000,0.000000,33.097706,-8.532648e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,36965.500000,0.000000,370.000000,34.919000,-5.026709e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55695.750000,20.000000,1320.000000,37.183936,-3.328922e+00,0.000000,17.000000,5.000000,213.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,15300.000000,2013.000000


In [ ]:
#Categorical Features Description
x_train.describe(include="O")

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,41580,39053,39044,41580,41580,41316,41580,41580,41580,39226,...,41580,41580,41580,41580,41580,41580,41580,41580,41580,41580
unique,346,1584,1803,27336,9,16012,21,125,2066,2,...,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Iringa,Njombe,Igosi,True,...,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,413,6333,12234,2535,7125,357,3737,1741,214,35632,...,17785,35531,35531,23197,23197,11863,11863,32075,19942,24194


In [ ]:
# Description of Feature "num_private" not provided in the DataDriven. Hence, checking for its value_counts.
x_train.num_private.value_counts(normalize=True,dropna=False).head(20)

0      0.986845
1      0.001251
6      0.001058
8      0.000866
32     0.000818
5      0.000794
15     0.000673
39     0.000625
45     0.000577
93     0.000577
7      0.000553
2      0.000433
3      0.000433
47     0.000409
4      0.000361
102    0.000337
65     0.000289
80     0.000289
20     0.000241
25     0.000216
Name: num_private, dtype: float64

In [ ]:
# Checking value_counts for Feature: "recorded_by"
x_train.recorded_by.value_counts(normalize=True,dropna=False)

GeoData Consultants Ltd    1.0
Name: recorded_by, dtype: float64

In [ ]:
# Checking datatype and unique values of each feature
def dtype_null(i):
    dt=[]
    n=[]
    u=[]
    for a in i.columns:
        dt.append(np.dtype(i[a]))
        n.append(i[a].isna().sum())
        u.append(i[a].nunique())
    fea_des=np.column_stack((i.columns,dt,n,u))
    fea_des=pd.DataFrame(fea_des,columns=["Feature","dtype","null","unique"])
    print(fea_des)
dtype_null(x_train)

                  Feature    dtype   null unique
0                      id    int64      0  41580
1              amount_tsh  float64      0     94
2           date_recorded   object      0    346
3                  funder   object   2527   1584
4              gps_height    int64      0   2383
5               installer   object   2536   1803
6               longitude  float64      0  40313
7                latitude  float64      0  40314
8                wpt_name   object      0  27336
9             num_private    int64      0     57
10                  basin   object      0      9
11             subvillage   object    264  16012
12                 region   object      0     21
13            region_code    int64      0     27
14          district_code    int64      0     20
15                    lga   object      0    125
16                   ward   object      0   2066
17             population    int64      0    927
18         public_meeting   object   2354      2
19            record

## Data Processing

In [ ]:
# making a copy of x_train and y_train. All processing will be done on the copies.

In [ ]:
x_train_2=x_train
y_train_2=y_train

In [ ]:
def process(a,b):
    a=a.drop(["id","recorded_by","extraction_type_group","extraction_type_class","management_group","water_quality","quantity_group","source","source_class","waterpoint_type_group","num_private"],axis=1)
    b=b.drop("id",axis=1)
    a["amount_tsh_cat"] = a["amount_tsh"].apply(lambda x: 0 if x == 0 else 1)
    date_format = '%Y-%m-%d'
    a["date_recorded"]=pd.to_datetime(a["date_recorded"])
    a["dsr"]=pd.to_datetime("2023-11-16")-a["date_recorded"]
    a.dsr=(a.dsr/ pd.Timedelta(seconds=86400)).astype("O")
    bin_edges=[np.min(a.gps_height)-1,0,500,1000,1500,2000,np.max(a.gps_height)]
    bin_labels=["<0","0-500","500-1000","1000-1500","1500-2000",">2000"]
    a["gps_height_cat"]=pd.cut(a.gps_height,bins=bin_edges,labels=bin_labels).astype("O")
    a["dst"]=np.sqrt(a["latitude"]**2+a["longitude"]**2)
    bin_edges=[np.min(a.construction_year)-1,0,1961,1971,1981,1991,2001,np.max(a.gps_height)+1]
    bin_labels=["0","0-1960","1960-70","1970-80","1980-90","1990-2000","2001-2013"]
    a["construction_year_cat"]=pd.cut(a.construction_year,bins=bin_edges,labels=bin_labels).astype("O")
    a=a.drop(["amount_tsh","date_recorded","gps_height","latitude","longitude","construction_year"],axis=1)
    a[["region_code","district_code","amount_tsh_cat"]]=a[["region_code","district_code","amount_tsh_cat"]].astype("O")
    cat=a.select_dtypes(include="O").columns
    a[cat] = a[cat].fillna('missing')
    a[["region_code","district_code","amount_tsh_cat"]]=a[["region_code","district_code","amount_tsh_cat"]].astype("O")
    return a,b


In [ ]:
x_train_2,y_train_2=process(x_train_2,y_train_2)

In [ ]:
x_train_2.dtypes

funder                    object
installer                 object
wpt_name                  object
basin                     object
subvillage                object
region                    object
region_code               object
district_code             object
lga                       object
ward                      object
population                 int64
public_meeting            object
scheme_management         object
scheme_name               object
permit                    object
extraction_type           object
management                object
payment                   object
payment_type              object
quality_group             object
quantity                  object
source_type               object
waterpoint_type           object
amount_tsh_cat            object
dsr                      float64
gps_height_cat            object
dst                      float64
construction_year_cat     object
dtype: object

In [ ]:
x_test_2,y_test_2=process(x_test,y_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
def matrix(xtr,ytr,xte,yte,est):
    print ("\n Train accuracy",round(accuracy_score(ytr, est.predict(xtr)),3))
    print ("\n Train Classification Report\n\n", classification_report(ytr, est.predict(xtr)))
    print ("\n Test accuracy", round(accuracy_score(yte, est.predict(xte)),3))
    print ("\n Test Classification Report\n\n", classification_report(yte, est.predict(xte)))

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
y_train_2

,status_group
43069,non functional
58858,functional
41707,non functional
30674,functional
20944,functional
...,...
54343,functional
38158,functional
860,non functional
15795,functional


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)
clf = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col)
parameters={"iterations":np.arange(10,110,10),"depth":np.arange(10,120,20),"learning_rate":[0.1]}
Grid=GridSearchCV(clf,param_grid=parameters,cv=10,scoring="f1_weighted")

In [ ]:
Grid.fit(x_train_2,y_train_2)

0:	learn: 1.0248153	total: 212ms	remaining: 1.91s
1:	learn: 0.9669819	total: 420ms	remaining: 1.68s
2:	learn: 0.9185694	total: 620ms	remaining: 1.45s
3:	learn: 0.8767320	total: 827ms	remaining: 1.24s
4:	learn: 0.8420713	total: 1.03s	remaining: 1.03s
5:	learn: 0.8126578	total: 1.21s	remaining: 809ms
6:	learn: 0.7874836	total: 1.42s	remaining: 606ms
7:	learn: 0.7636150	total: 1.64s	remaining: 411ms
8:	learn: 0.7420164	total: 1.84s	remaining: 205ms
9:	learn: 0.7238264	total: 2.06s	remaining: 0us
0:	learn: 1.0247878	total: 202ms	remaining: 1.82s
1:	learn: 0.9655781	total: 393ms	remaining: 1.57s
2:	learn: 0.9167279	total: 583ms	remaining: 1.36s
3:	learn: 0.8750651	total: 771ms	remaining: 1.16s
4:	learn: 0.8404164	total: 956ms	remaining: 956ms
5:	learn: 0.8112060	total: 1.14s	remaining: 761ms
6:	learn: 0.7857892	total: 1.33s	remaining: 570ms
7:	learn: 0.7638486	total: 1.51s	remaining: 379ms
8:	learn: 0.7429936	total: 1.71s	remaining: 190ms
9:	learn: 0.7255384	total: 1.94s	remaining: 0us
0:	l

6:	learn: 0.7842468	total: 1.38s	remaining: 2.56s
7:	learn: 0.7604609	total: 1.56s	remaining: 2.35s
8:	learn: 0.7418175	total: 1.67s	remaining: 2.04s
9:	learn: 0.7241798	total: 1.87s	remaining: 1.87s
10:	learn: 0.7081962	total: 2.05s	remaining: 1.68s
11:	learn: 0.6948862	total: 2.24s	remaining: 1.49s
12:	learn: 0.6825041	total: 2.43s	remaining: 1.31s
13:	learn: 0.6717361	total: 2.62s	remaining: 1.12s
14:	learn: 0.6618280	total: 2.82s	remaining: 939ms
15:	learn: 0.6511959	total: 3s	remaining: 751ms
16:	learn: 0.6427322	total: 3.21s	remaining: 567ms
17:	learn: 0.6354652	total: 3.42s	remaining: 381ms
18:	learn: 0.6288555	total: 3.62s	remaining: 190ms
19:	learn: 0.6219958	total: 3.83s	remaining: 0us
0:	learn: 1.0243612	total: 202ms	remaining: 3.84s
1:	learn: 0.9653196	total: 409ms	remaining: 3.68s
2:	learn: 0.9162810	total: 607ms	remaining: 3.44s
3:	learn: 0.8747885	total: 800ms	remaining: 3.2s
4:	learn: 0.8391424	total: 1s	remaining: 3s
5:	learn: 0.8098040	total: 1.19s	remaining: 2.77s
6:

29:	learn: 0.5820941	total: 6s	remaining: 0us
0:	learn: 1.0247878	total: 200ms	remaining: 5.79s
1:	learn: 0.9655781	total: 405ms	remaining: 5.67s
2:	learn: 0.9167279	total: 642ms	remaining: 5.78s
3:	learn: 0.8750651	total: 835ms	remaining: 5.42s
4:	learn: 0.8404164	total: 1.03s	remaining: 5.17s
5:	learn: 0.8112060	total: 1.23s	remaining: 4.94s
6:	learn: 0.7857892	total: 1.44s	remaining: 4.72s
7:	learn: 0.7638486	total: 1.64s	remaining: 4.5s
8:	learn: 0.7429936	total: 1.84s	remaining: 4.28s
9:	learn: 0.7255384	total: 2.03s	remaining: 4.06s
10:	learn: 0.7097105	total: 2.23s	remaining: 3.85s
11:	learn: 0.6955509	total: 2.44s	remaining: 3.66s
12:	learn: 0.6849650	total: 2.64s	remaining: 3.45s
13:	learn: 0.6735093	total: 2.84s	remaining: 3.24s
14:	learn: 0.6634603	total: 3.03s	remaining: 3.03s
15:	learn: 0.6528971	total: 3.23s	remaining: 2.82s
16:	learn: 0.6447852	total: 3.42s	remaining: 2.61s
17:	learn: 0.6375906	total: 3.62s	remaining: 2.41s
18:	learn: 0.6305907	total: 3.82s	remaining: 2.

12:	learn: 0.6821633	total: 2.48s	remaining: 3.25s
13:	learn: 0.6710418	total: 2.68s	remaining: 3.06s
14:	learn: 0.6609993	total: 2.88s	remaining: 2.88s
15:	learn: 0.6533344	total: 3.07s	remaining: 2.68s
16:	learn: 0.6451859	total: 3.27s	remaining: 2.5s
17:	learn: 0.6379922	total: 3.48s	remaining: 2.32s
18:	learn: 0.6320045	total: 3.68s	remaining: 2.13s
19:	learn: 0.6253564	total: 3.88s	remaining: 1.94s
20:	learn: 0.6183322	total: 4.08s	remaining: 1.75s
21:	learn: 0.6129823	total: 4.28s	remaining: 1.55s
22:	learn: 0.6069662	total: 4.47s	remaining: 1.36s
23:	learn: 0.6028135	total: 4.67s	remaining: 1.17s
24:	learn: 0.5986444	total: 4.86s	remaining: 972ms
25:	learn: 0.5946332	total: 5.05s	remaining: 777ms
26:	learn: 0.5903212	total: 5.25s	remaining: 583ms
27:	learn: 0.5864888	total: 5.45s	remaining: 389ms
28:	learn: 0.5827389	total: 5.65s	remaining: 195ms
29:	learn: 0.5800981	total: 5.84s	remaining: 0us
0:	learn: 1.0223011	total: 207ms	remaining: 6.01s
1:	learn: 0.9622286	total: 409ms	re

16:	learn: 0.6447852	total: 3.34s	remaining: 4.52s
17:	learn: 0.6375906	total: 3.54s	remaining: 4.33s
18:	learn: 0.6305907	total: 3.74s	remaining: 4.13s
19:	learn: 0.6240923	total: 3.95s	remaining: 3.95s
20:	learn: 0.6180444	total: 4.14s	remaining: 3.75s
21:	learn: 0.6127301	total: 4.34s	remaining: 3.55s
22:	learn: 0.6073902	total: 4.53s	remaining: 3.35s
23:	learn: 0.6035623	total: 4.72s	remaining: 3.15s
24:	learn: 0.5995346	total: 4.92s	remaining: 2.95s
25:	learn: 0.5944531	total: 5.12s	remaining: 2.76s
26:	learn: 0.5913329	total: 5.32s	remaining: 2.56s
27:	learn: 0.5876758	total: 5.53s	remaining: 2.37s
28:	learn: 0.5846884	total: 5.73s	remaining: 2.17s
29:	learn: 0.5810587	total: 5.92s	remaining: 1.97s
30:	learn: 0.5776842	total: 6.13s	remaining: 1.78s
31:	learn: 0.5740788	total: 6.33s	remaining: 1.58s
32:	learn: 0.5710644	total: 6.53s	remaining: 1.38s
33:	learn: 0.5679820	total: 6.73s	remaining: 1.19s
34:	learn: 0.5650157	total: 6.93s	remaining: 990ms
35:	learn: 0.5625172	total: 7.1

...

18:	learn: 0.6292249	total: 3.66s	remaining: 15.6s
19:	learn: 0.6229493	total: 3.85s	remaining: 15.4s
20:	learn: 0.6178411	total: 4.04s	remaining: 15.2s
21:	learn: 0.6130577	total: 4.24s	remaining: 15s
22:	learn: 0.6083633	total: 4.45s	remaining: 14.9s
23:	learn: 0.6040271	total: 4.64s	remaining: 14.7s
24:	learn: 0.5992274	total: 4.83s	remaining: 14.5s
25:	learn: 0.5947075	total: 5.03s	remaining: 14.3s
26:	learn: 0.5910118	total: 5.24s	remaining: 14.2s
27:	learn: 0.5877140	total: 5.45s	remaining: 14s
28:	learn: 0.5845904	total: 5.63s	remaining: 13.8s
29:	learn: 0.5811757	total: 5.84s	remaining: 13.6s
30:	learn: 0.5775692	total: 6.03s	remaining: 13.4s
31:	learn: 0.5736120	total: 6.22s	remaining: 13.2s
32:	learn: 0.5704567	total: 6.42s	remaining: 13s
33:	learn: 0.5676319	total: 6.62s	remaining: 12.8s
34:	learn: 0.5643933	total: 6.82s	remaining: 12.7s
35:	learn: 0.5616173	total: 7.02s	remaining: 12.5s
36:	learn: 0.5593793	total: 7.21s	remaining: 12.3s
37:	learn: 0.5572323	total: 7.41s	rem

80:	learn: 0.4959336	total: 17.3s	remaining: 4.06s
81:	learn: 0.4947221	total: 17.5s	remaining: 3.85s
82:	learn: 0.4932059	total: 17.7s	remaining: 3.63s
83:	learn: 0.4922868	total: 17.9s	remaining: 3.41s
84:	learn: 0.4917852	total: 18.1s	remaining: 3.2s
85:	learn: 0.4899729	total: 18.3s	remaining: 2.98s
86:	learn: 0.4885809	total: 18.5s	remaining: 2.76s
87:	learn: 0.4874297	total: 18.7s	remaining: 2.55s
88:	learn: 0.4864113	total: 18.9s	remaining: 2.33s
89:	learn: 0.4855636	total: 19.1s	remaining: 2.12s
90:	learn: 0.4848827	total: 19.3s	remaining: 1.91s
91:	learn: 0.4825683	total: 19.5s	remaining: 1.7s
92:	learn: 0.4814205	total: 19.7s	remaining: 1.48s
93:	learn: 0.4802600	total: 19.9s	remaining: 1.27s
94:	learn: 0.4791976	total: 20.1s	remaining: 1.06s
95:	learn: 0.4784882	total: 20.3s	remaining: 847ms
96:	learn: 0.4776580	total: 20.5s	remaining: 635ms
97:	learn: 0.4768984	total: 20.7s	remaining: 423ms
98:	learn: 0.4761457	total: 20.9s	remaining: 211ms
99:	learn: 0.4756813	total: 21.1s

C:\Users\siddh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
500 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\siddh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\siddh\anaconda3\Lib\site-packages\catboost\core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "C:\Users\siddh\anaconda3\Lib\site-packages\catboost\core.py", line 2341

0:	learn: 1.0240479	total: 235ms	remaining: 23.3s
1:	learn: 0.9646023	total: 446ms	remaining: 21.9s
2:	learn: 0.9159819	total: 649ms	remaining: 21s
3:	learn: 0.8748921	total: 846ms	remaining: 20.3s
4:	learn: 0.8392158	total: 1.05s	remaining: 20s
5:	learn: 0.8097586	total: 1.25s	remaining: 19.6s
6:	learn: 0.7850181	total: 1.45s	remaining: 19.2s
7:	learn: 0.7604636	total: 1.64s	remaining: 18.9s
8:	learn: 0.7396203	total: 1.84s	remaining: 18.6s
9:	learn: 0.7241478	total: 1.93s	remaining: 17.4s
10:	learn: 0.7073687	total: 2.13s	remaining: 17.3s
11:	learn: 0.6930091	total: 2.33s	remaining: 17.1s
12:	learn: 0.6804533	total: 2.53s	remaining: 16.9s
13:	learn: 0.6693012	total: 2.73s	remaining: 16.8s
14:	learn: 0.6591169	total: 2.93s	remaining: 16.6s
15:	learn: 0.6502731	total: 3.12s	remaining: 16.4s
16:	learn: 0.6420922	total: 3.32s	remaining: 16.2s
17:	learn: 0.6349124	total: 3.51s	remaining: 16s
18:	learn: 0.6278605	total: 3.71s	remaining: 15.8s
19:	learn: 0.6215435	total: 3.9s	remaining: 15.

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostClassifier object at 0x00000239C66E1B10>,
             param_grid={'depth': array([ 10,  30,  50,  70,  90, 110]),
                         'iterations': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]),
                         'learning_rate': [0.1]},
             scoring='f1_weighted')

In [ ]:
Grid.best_params_

{'depth': 10, 'iterations': 100, 'learning_rate': 0.1}

In [ ]:
Grid.best_estimator_

In [ ]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Grid)


 Train accuracy 0.829

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.94      0.87     22535
functional needs repair       0.75      0.28      0.41      3024
         non functional       0.89      0.77      0.83     16021

               accuracy                           0.83     41580
              macro avg       0.81      0.67      0.70     41580
           weighted avg       0.83      0.83      0.82     41580


 Test accuracy 0.793

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.77      0.92      0.84      9724
functional needs repair       0.67      0.22      0.33      1293
         non functional       0.84      0.73      0.78      6803

               accuracy                           0.79     17820
              macro avg       0.76      0.62      0.65     17820
           weighted avg       0.79      0.

In [ ]:
Grid.best_estimator_

NameError: name 'Grid' is not defined

## Using GridSearch CV

In [ ]:
from sklearn.model_selection import GridSearchCV
Cat = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col,max_depth=10)
parameters={"iterations":np.arange(100,400,50),"learning_rate":[0.05,1.1,0.1]}
Grid=GridSearchCV(clf,param_grid=parameters,cv=10,scoring="f1_weighted")

In [ ]:
Grid.fit(x_train_2,y_train_2)

0:	learn: 1.0607619	total: 282ms	remaining: 27.9s
1:	learn: 1.0274759	total: 568ms	remaining: 27.9s
2:	learn: 0.9968490	total: 806ms	remaining: 26s
3:	learn: 0.9689932	total: 1.03s	remaining: 24.8s
4:	learn: 0.9446711	total: 1.25s	remaining: 23.9s
5:	learn: 0.9224064	total: 1.48s	remaining: 23.2s
6:	learn: 0.9014378	total: 1.71s	remaining: 22.7s
7:	learn: 0.8822688	total: 1.96s	remaining: 22.5s
8:	learn: 0.8636448	total: 2.19s	remaining: 22.1s
9:	learn: 0.8463714	total: 2.43s	remaining: 21.8s
10:	learn: 0.8304303	total: 2.66s	remaining: 21.5s
11:	learn: 0.8159682	total: 2.89s	remaining: 21.2s
12:	learn: 0.8022914	total: 3.12s	remaining: 20.9s
13:	learn: 0.7894770	total: 3.38s	remaining: 20.7s
14:	learn: 0.7779313	total: 3.6s	remaining: 20.4s
15:	learn: 0.7667527	total: 3.81s	remaining: 20s
16:	learn: 0.7562529	total: 4.03s	remaining: 19.7s
17:	learn: 0.7456847	total: 4.27s	remaining: 19.4s
18:	learn: 0.7363421	total: 4.52s	remaining: 19.3s
19:	learn: 0.7278483	total: 4.77s	remaining: 1

62:	learn: 0.5750893	total: 14.4s	remaining: 8.47s
63:	learn: 0.5739346	total: 14.6s	remaining: 8.24s
64:	learn: 0.5723109	total: 14.9s	remaining: 8.01s
65:	learn: 0.5709480	total: 15.1s	remaining: 7.78s
66:	learn: 0.5694817	total: 15.3s	remaining: 7.55s
67:	learn: 0.5679235	total: 15.6s	remaining: 7.32s
68:	learn: 0.5666231	total: 15.8s	remaining: 7.09s
69:	learn: 0.5653060	total: 16s	remaining: 6.86s
70:	learn: 0.5638719	total: 16.2s	remaining: 6.63s
71:	learn: 0.5624237	total: 16.5s	remaining: 6.4s
72:	learn: 0.5612927	total: 16.7s	remaining: 6.17s
73:	learn: 0.5601388	total: 16.9s	remaining: 5.93s
74:	learn: 0.5591353	total: 17.1s	remaining: 5.71s
75:	learn: 0.5576116	total: 17.3s	remaining: 5.48s
76:	learn: 0.5564562	total: 17.6s	remaining: 5.25s
77:	learn: 0.5554692	total: 17.8s	remaining: 5.02s
78:	learn: 0.5540321	total: 18.1s	remaining: 4.8s
79:	learn: 0.5529646	total: 18.3s	remaining: 4.58s
80:	learn: 0.5518554	total: 18.6s	remaining: 4.36s
81:	learn: 0.5509559	total: 18.9s	r

24:	learn: 0.6890706	total: 6.11s	remaining: 18.3s
25:	learn: 0.6829637	total: 6.36s	remaining: 18.1s
26:	learn: 0.6772031	total: 6.63s	remaining: 17.9s
27:	learn: 0.6718217	total: 6.9s	remaining: 17.7s
28:	learn: 0.6667726	total: 7.15s	remaining: 17.5s
29:	learn: 0.6613855	total: 7.39s	remaining: 17.2s
30:	learn: 0.6565540	total: 7.63s	remaining: 17s
31:	learn: 0.6520026	total: 7.94s	remaining: 16.9s
32:	learn: 0.6475092	total: 8.22s	remaining: 16.7s
33:	learn: 0.6434877	total: 8.49s	remaining: 16.5s
34:	learn: 0.6397822	total: 8.76s	remaining: 16.3s
35:	learn: 0.6357018	total: 9.01s	remaining: 16s
36:	learn: 0.6319545	total: 9.26s	remaining: 15.8s
37:	learn: 0.6284699	total: 9.49s	remaining: 15.5s
38:	learn: 0.6252927	total: 9.75s	remaining: 15.3s
39:	learn: 0.6227536	total: 10s	remaining: 15s
40:	learn: 0.6199838	total: 10.3s	remaining: 14.8s
41:	learn: 0.6171306	total: 10.5s	remaining: 14.5s
42:	learn: 0.6143928	total: 10.8s	remaining: 14.3s
43:	learn: 0.6115529	total: 11s	remainin

86:	learn: 0.5470650	total: 20.4s	remaining: 3.04s
87:	learn: 0.5458151	total: 20.7s	remaining: 2.82s
88:	learn: 0.5450411	total: 20.9s	remaining: 2.58s
89:	learn: 0.5441593	total: 21.1s	remaining: 2.35s
90:	learn: 0.5432408	total: 21.4s	remaining: 2.12s
91:	learn: 0.5422543	total: 21.6s	remaining: 1.88s
92:	learn: 0.5416009	total: 21.9s	remaining: 1.65s
93:	learn: 0.5409381	total: 22.1s	remaining: 1.41s
94:	learn: 0.5401797	total: 22.3s	remaining: 1.18s
95:	learn: 0.5392912	total: 22.6s	remaining: 940ms
96:	learn: 0.5385988	total: 22.8s	remaining: 705ms
97:	learn: 0.5376657	total: 23s	remaining: 470ms
98:	learn: 0.5366685	total: 23.2s	remaining: 235ms
99:	learn: 0.5357432	total: 23.5s	remaining: 0us
0:	learn: 1.0602152	total: 248ms	remaining: 24.5s
1:	learn: 1.0260998	total: 487ms	remaining: 23.9s
2:	learn: 0.9956900	total: 712ms	remaining: 23s
3:	learn: 0.9676942	total: 927ms	remaining: 22.2s
4:	learn: 0.9426955	total: 1.14s	remaining: 21.7s
5:	learn: 0.9197540	total: 1.36s	remaining

49:	learn: 0.5974936	total: 11.6s	remaining: 11.6s
50:	learn: 0.5954618	total: 11.9s	remaining: 11.4s
51:	learn: 0.5936925	total: 12.2s	remaining: 11.2s
52:	learn: 0.5921254	total: 12.4s	remaining: 11s
53:	learn: 0.5903228	total: 12.7s	remaining: 10.8s
54:	learn: 0.5884100	total: 12.9s	remaining: 10.6s
55:	learn: 0.5862377	total: 13.2s	remaining: 10.4s
56:	learn: 0.5857542	total: 13.2s	remaining: 9.98s
57:	learn: 0.5840689	total: 13.5s	remaining: 9.77s
58:	learn: 0.5824141	total: 13.8s	remaining: 9.56s
59:	learn: 0.5809676	total: 14s	remaining: 9.33s
60:	learn: 0.5792019	total: 14.2s	remaining: 9.1s
61:	learn: 0.5774308	total: 14.5s	remaining: 8.88s
62:	learn: 0.5759943	total: 14.7s	remaining: 8.64s
63:	learn: 0.5743397	total: 15s	remaining: 8.44s
64:	learn: 0.5729049	total: 15.2s	remaining: 8.2s
65:	learn: 0.5713301	total: 15.5s	remaining: 7.96s
66:	learn: 0.5697940	total: 15.7s	remaining: 7.73s
67:	learn: 0.5684831	total: 15.9s	remaining: 7.49s
68:	learn: 0.5669270	total: 16.1s	remai

...

81:	learn: 0.4733404	total: 1m 12s	remaining: 3m 57s
82:	learn: 0.4721373	total: 1m 13s	remaining: 3m 57s
83:	learn: 0.4708385	total: 1m 14s	remaining: 3m 56s
84:	learn: 0.4702448	total: 1m 15s	remaining: 3m 55s
85:	learn: 0.4692106	total: 1m 16s	remaining: 3m 55s
86:	learn: 0.4687363	total: 1m 17s	remaining: 3m 54s
87:	learn: 0.4683556	total: 1m 18s	remaining: 3m 53s
88:	learn: 0.4674996	total: 1m 19s	remaining: 3m 53s
89:	learn: 0.4662660	total: 1m 20s	remaining: 3m 52s
90:	learn: 0.4654562	total: 1m 21s	remaining: 3m 51s
91:	learn: 0.4639981	total: 1m 22s	remaining: 3m 51s
92:	learn: 0.4628090	total: 1m 23s	remaining: 3m 50s
93:	learn: 0.4621198	total: 1m 24s	remaining: 3m 49s
94:	learn: 0.4613787	total: 1m 25s	remaining: 3m 48s
95:	learn: 0.4599805	total: 1m 26s	remaining: 3m 47s
96:	learn: 0.4593614	total: 1m 26s	remaining: 3m 46s
97:	learn: 0.4589876	total: 1m 27s	remaining: 3m 45s
98:	learn: 0.4586795	total: 1m 28s	remaining: 3m 45s
99:	learn: 0.4582437	total: 1m 29s	remaining: 

235:	learn: 0.3805354	total: 3m 36s	remaining: 1m 44s
236:	learn: 0.3802192	total: 3m 37s	remaining: 1m 43s
237:	learn: 0.3798524	total: 3m 38s	remaining: 1m 42s
238:	learn: 0.3794786	total: 3m 39s	remaining: 1m 41s
239:	learn: 0.3786905	total: 3m 40s	remaining: 1m 41s
240:	learn: 0.3784258	total: 3m 41s	remaining: 1m 40s
241:	learn: 0.3783429	total: 3m 42s	remaining: 1m 39s
242:	learn: 0.3780838	total: 3m 43s	remaining: 1m 38s
243:	learn: 0.3774745	total: 3m 44s	remaining: 1m 37s
244:	learn: 0.3766925	total: 3m 45s	remaining: 1m 36s
245:	learn: 0.3763138	total: 3m 46s	remaining: 1m 35s
246:	learn: 0.3758196	total: 3m 47s	remaining: 1m 34s
247:	learn: 0.3752207	total: 3m 48s	remaining: 1m 33s
248:	learn: 0.3746312	total: 3m 49s	remaining: 1m 32s
249:	learn: 0.3743494	total: 3m 49s	remaining: 1m 31s
250:	learn: 0.3738899	total: 3m 50s	remaining: 1m 31s
251:	learn: 0.3737113	total: 3m 51s	remaining: 1m 30s
252:	learn: 0.3730159	total: 3m 52s	remaining: 1m 29s
253:	learn: 0.3726078	total:

42:	learn: 0.5184651	total: 41.1s	remaining: 4m 53s
43:	learn: 0.5164925	total: 42.2s	remaining: 4m 53s
44:	learn: 0.5152815	total: 43.3s	remaining: 4m 53s
45:	learn: 0.5129232	total: 44.3s	remaining: 4m 52s
46:	learn: 0.5119752	total: 45.2s	remaining: 4m 51s
47:	learn: 0.5095643	total: 46.1s	remaining: 4m 50s
48:	learn: 0.5076583	total: 47s	remaining: 4m 48s
49:	learn: 0.5063286	total: 48.1s	remaining: 4m 48s
50:	learn: 0.5042977	total: 49.1s	remaining: 4m 47s
51:	learn: 0.5033591	total: 50.1s	remaining: 4m 47s
52:	learn: 0.5019387	total: 51.2s	remaining: 4m 46s
53:	learn: 0.5007009	total: 52.3s	remaining: 4m 46s
54:	learn: 0.4998740	total: 53s	remaining: 4m 44s
55:	learn: 0.4983828	total: 54.1s	remaining: 4m 43s
56:	learn: 0.4972579	total: 55.2s	remaining: 4m 43s
57:	learn: 0.4960871	total: 56.2s	remaining: 4m 42s
58:	learn: 0.4952594	total: 57.1s	remaining: 4m 41s
59:	learn: 0.4941659	total: 58.1s	remaining: 4m 40s
60:	learn: 0.4932531	total: 59.2s	remaining: 4m 40s
61:	learn: 0.491

197:	learn: 0.4019647	total: 3m 20s	remaining: 2m 33s
198:	learn: 0.4018236	total: 3m 21s	remaining: 2m 32s
199:	learn: 0.4014686	total: 3m 22s	remaining: 2m 31s
200:	learn: 0.4011836	total: 3m 23s	remaining: 2m 30s
201:	learn: 0.4005713	total: 3m 24s	remaining: 2m 29s
202:	learn: 0.4002921	total: 3m 25s	remaining: 2m 28s
203:	learn: 0.3999803	total: 3m 26s	remaining: 2m 27s
204:	learn: 0.3994364	total: 3m 27s	remaining: 2m 26s
205:	learn: 0.3991886	total: 3m 28s	remaining: 2m 25s
206:	learn: 0.3990248	total: 3m 29s	remaining: 2m 24s
207:	learn: 0.3987539	total: 3m 30s	remaining: 2m 23s
208:	learn: 0.3980497	total: 3m 31s	remaining: 2m 22s
209:	learn: 0.3978401	total: 3m 32s	remaining: 2m 21s
210:	learn: 0.3975366	total: 3m 33s	remaining: 2m 20s
211:	learn: 0.3971329	total: 3m 34s	remaining: 2m 19s
212:	learn: 0.3962264	total: 3m 35s	remaining: 2m 18s
213:	learn: 0.3956271	total: 3m 36s	remaining: 2m 17s
214:	learn: 0.3950344	total: 3m 38s	remaining: 2m 16s
215:	learn: 0.3943709	total:

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001AF38D33410>,
             param_grid={'iterations': array([100, 150, 200, 250, 300, 350]),
                         'learning_rate': [0.05, 1.1, 0.1]},
             scoring='f1_weighted')

In [ ]:
Grid.best_params_

{'iterations': 350, 'learning_rate': 0.1}

In [ ]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Grid)


 Train accuracy 0.88

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.86      0.96      0.90     22535
functional needs repair       0.79      0.43      0.56      3024
         non functional       0.93      0.86      0.89     16021

               accuracy                           0.88     41580
              macro avg       0.86      0.75      0.79     41580
           weighted avg       0.88      0.88      0.87     41580


 Test accuracy 0.81

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9724
functional needs repair       0.62      0.30      0.41      1293
         non functional       0.85      0.77      0.81      6803

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.81      0.81

In [ ]:
cat_col=list(x_train_2.select_dtypes(include="O").columns)
Cat_8127 = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col,max_depth=10,iterations=350,learning_rate= 0.1)
Cat_8127.fit(x_train_2,y_train_2)

0:	learn: 1.0187846	total: 868ms	remaining: 5m 2s
1:	learn: 0.9543063	total: 1.56s	remaining: 4m 30s
2:	learn: 0.9030945	total: 2.3s	remaining: 4m 26s
3:	learn: 0.8587966	total: 2.9s	remaining: 4m 10s
4:	learn: 0.8213831	total: 3.58s	remaining: 4m 7s
5:	learn: 0.7898780	total: 4.2s	remaining: 4m
6:	learn: 0.7640097	total: 4.89s	remaining: 3m 59s
7:	learn: 0.7392438	total: 5.53s	remaining: 3m 56s
8:	learn: 0.7193832	total: 6.23s	remaining: 3m 56s
9:	learn: 0.7014090	total: 6.95s	remaining: 3m 56s
10:	learn: 0.6853621	total: 7.63s	remaining: 3m 55s
11:	learn: 0.6704419	total: 8.35s	remaining: 3m 55s
12:	learn: 0.6576645	total: 8.89s	remaining: 3m 50s
13:	learn: 0.6455592	total: 9.51s	remaining: 3m 48s
14:	learn: 0.6335305	total: 10.1s	remaining: 3m 45s
15:	learn: 0.6228480	total: 10.8s	remaining: 3m 45s
16:	learn: 0.6141860	total: 11.5s	remaining: 3m 46s
17:	learn: 0.6055604	total: 12.2s	remaining: 3m 45s
18:	learn: 0.5986696	total: 12.9s	remaining: 3m 45s
19:	learn: 0.5928729	total: 13.

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_scores = cross_val_score(Cat_8127, x_train_2, y_train_2.values.ravel(), cv=5, scoring="accuracy")

0:	learn: 1.0186521	total: 584ms	remaining: 3m 23s
1:	learn: 0.9569389	total: 909ms	remaining: 2m 38s
2:	learn: 0.9044003	total: 1.54s	remaining: 2m 57s
3:	learn: 0.8605886	total: 2.04s	remaining: 2m 56s
4:	learn: 0.8248893	total: 2.59s	remaining: 2m 58s
5:	learn: 0.7945476	total: 3.23s	remaining: 3m 5s
6:	learn: 0.7704371	total: 3.82s	remaining: 3m 7s
7:	learn: 0.7466592	total: 4.42s	remaining: 3m 9s
8:	learn: 0.7269339	total: 5.13s	remaining: 3m 14s
9:	learn: 0.7103245	total: 5.82s	remaining: 3m 17s
10:	learn: 0.6934453	total: 6.43s	remaining: 3m 18s
11:	learn: 0.6787891	total: 7.16s	remaining: 3m 21s
12:	learn: 0.6659185	total: 7.93s	remaining: 3m 25s
13:	learn: 0.6548574	total: 8.63s	remaining: 3m 27s
14:	learn: 0.6443328	total: 9.31s	remaining: 3m 27s
15:	learn: 0.6347563	total: 10s	remaining: 3m 29s
16:	learn: 0.6265772	total: 10.8s	remaining: 3m 31s
17:	learn: 0.6183476	total: 11.4s	remaining: 3m 30s
18:	learn: 0.6110483	total: 12s	remaining: 3m 29s
19:	learn: 0.6050728	total: 1

157:	learn: 0.4207366	total: 1m 57s	remaining: 2m 23s
158:	learn: 0.4203149	total: 1m 58s	remaining: 2m 22s
159:	learn: 0.4192959	total: 1m 59s	remaining: 2m 21s
160:	learn: 0.4186799	total: 2m	remaining: 2m 21s
161:	learn: 0.4184407	total: 2m	remaining: 2m 20s
162:	learn: 0.4179711	total: 2m 1s	remaining: 2m 19s
163:	learn: 0.4164481	total: 2m 2s	remaining: 2m 18s
164:	learn: 0.4155809	total: 2m 3s	remaining: 2m 18s
165:	learn: 0.4151594	total: 2m 3s	remaining: 2m 17s
166:	learn: 0.4148738	total: 2m 4s	remaining: 2m 16s
167:	learn: 0.4143416	total: 2m 5s	remaining: 2m 15s
168:	learn: 0.4140170	total: 2m 6s	remaining: 2m 15s
169:	learn: 0.4137196	total: 2m 6s	remaining: 2m 14s
170:	learn: 0.4128525	total: 2m 7s	remaining: 2m 13s
171:	learn: 0.4122874	total: 2m 8s	remaining: 2m 12s
172:	learn: 0.4117843	total: 2m 8s	remaining: 2m 11s
173:	learn: 0.4113678	total: 2m 9s	remaining: 2m 11s
174:	learn: 0.4110377	total: 2m 10s	remaining: 2m 10s
175:	learn: 0.4106921	total: 2m 11s	remaining: 2

312:	learn: 0.3459935	total: 3m 49s	remaining: 27.1s
313:	learn: 0.3459043	total: 3m 49s	remaining: 26.3s
314:	learn: 0.3452111	total: 3m 50s	remaining: 25.6s
315:	learn: 0.3451402	total: 3m 50s	remaining: 24.8s
316:	learn: 0.3448120	total: 3m 51s	remaining: 24.1s
317:	learn: 0.3446067	total: 3m 52s	remaining: 23.4s
318:	learn: 0.3444475	total: 3m 52s	remaining: 22.6s
319:	learn: 0.3439774	total: 3m 53s	remaining: 21.9s
320:	learn: 0.3435958	total: 3m 54s	remaining: 21.2s
321:	learn: 0.3432797	total: 3m 55s	remaining: 20.4s
322:	learn: 0.3430050	total: 3m 55s	remaining: 19.7s
323:	learn: 0.3427236	total: 3m 56s	remaining: 19s
324:	learn: 0.3422895	total: 3m 57s	remaining: 18.3s
325:	learn: 0.3418887	total: 3m 58s	remaining: 17.5s
326:	learn: 0.3417465	total: 3m 58s	remaining: 16.8s
327:	learn: 0.3413874	total: 3m 59s	remaining: 16.1s
328:	learn: 0.3409664	total: 4m	remaining: 15.3s
329:	learn: 0.3405373	total: 4m	remaining: 14.6s
330:	learn: 0.3402753	total: 4m 1s	remaining: 13.9s
331:

120:	learn: 0.4411928	total: 1m 26s	remaining: 2m 42s
121:	learn: 0.4407218	total: 1m 26s	remaining: 2m 42s
122:	learn: 0.4401729	total: 1m 27s	remaining: 2m 41s
123:	learn: 0.4397956	total: 1m 28s	remaining: 2m 40s
124:	learn: 0.4390839	total: 1m 28s	remaining: 2m 39s
125:	learn: 0.4389025	total: 1m 29s	remaining: 2m 39s
126:	learn: 0.4380555	total: 1m 30s	remaining: 2m 38s
127:	learn: 0.4373621	total: 1m 31s	remaining: 2m 37s
128:	learn: 0.4365026	total: 1m 31s	remaining: 2m 37s
129:	learn: 0.4362296	total: 1m 32s	remaining: 2m 36s
130:	learn: 0.4355913	total: 1m 33s	remaining: 2m 36s
131:	learn: 0.4341402	total: 1m 34s	remaining: 2m 35s
132:	learn: 0.4334571	total: 1m 34s	remaining: 2m 34s
133:	learn: 0.4323908	total: 1m 35s	remaining: 2m 34s
134:	learn: 0.4319534	total: 1m 36s	remaining: 2m 33s
135:	learn: 0.4316399	total: 1m 36s	remaining: 2m 32s
136:	learn: 0.4310601	total: 1m 37s	remaining: 2m 31s
137:	learn: 0.4299353	total: 1m 38s	remaining: 2m 30s
138:	learn: 0.4290144	total:

274:	learn: 0.3614336	total: 3m 17s	remaining: 53.8s
275:	learn: 0.3606222	total: 3m 18s	remaining: 53.1s
276:	learn: 0.3602532	total: 3m 18s	remaining: 52.4s
277:	learn: 0.3599246	total: 3m 19s	remaining: 51.7s
278:	learn: 0.3594328	total: 3m 20s	remaining: 51s
279:	learn: 0.3588626	total: 3m 21s	remaining: 50.3s
280:	learn: 0.3586785	total: 3m 21s	remaining: 49.6s
281:	learn: 0.3583614	total: 3m 22s	remaining: 48.8s
282:	learn: 0.3580694	total: 3m 23s	remaining: 48.2s
283:	learn: 0.3577787	total: 3m 24s	remaining: 47.5s
284:	learn: 0.3571404	total: 3m 24s	remaining: 46.7s
285:	learn: 0.3568443	total: 3m 25s	remaining: 46s
286:	learn: 0.3564291	total: 3m 26s	remaining: 45.3s
287:	learn: 0.3562160	total: 3m 27s	remaining: 44.6s
288:	learn: 0.3558902	total: 3m 27s	remaining: 43.8s
289:	learn: 0.3550242	total: 3m 28s	remaining: 43.1s
290:	learn: 0.3546463	total: 3m 29s	remaining: 42.4s
291:	learn: 0.3539872	total: 3m 29s	remaining: 41.7s
292:	learn: 0.3537096	total: 3m 30s	remaining: 41s

...

43:	learn: 0.5246803	total: 31.9s	remaining: 3m 42s
44:	learn: 0.5235385	total: 32.6s	remaining: 3m 41s
45:	learn: 0.5216755	total: 33.4s	remaining: 3m 40s
46:	learn: 0.5197544	total: 34.1s	remaining: 3m 39s
47:	learn: 0.5189681	total: 34.7s	remaining: 3m 38s
48:	learn: 0.5168564	total: 35.4s	remaining: 3m 37s
49:	learn: 0.5147879	total: 36.2s	remaining: 3m 37s
50:	learn: 0.5127387	total: 36.9s	remaining: 3m 36s
51:	learn: 0.5107525	total: 37.6s	remaining: 3m 35s
52:	learn: 0.5090543	total: 38.3s	remaining: 3m 34s
53:	learn: 0.5071774	total: 39s	remaining: 3m 33s
54:	learn: 0.5061264	total: 39.7s	remaining: 3m 32s
55:	learn: 0.5048037	total: 40.4s	remaining: 3m 32s
56:	learn: 0.5036134	total: 41.1s	remaining: 3m 31s
57:	learn: 0.5023478	total: 41.8s	remaining: 3m 30s
58:	learn: 0.5007710	total: 42.5s	remaining: 3m 29s
59:	learn: 0.4995052	total: 43.2s	remaining: 3m 29s
60:	learn: 0.4985216	total: 44s	remaining: 3m 28s
61:	learn: 0.4973687	total: 44.5s	remaining: 3m 26s
62:	learn: 0.496

199:	learn: 0.4029146	total: 2m 24s	remaining: 1m 48s
200:	learn: 0.4022732	total: 2m 25s	remaining: 1m 47s
201:	learn: 0.4018083	total: 2m 26s	remaining: 1m 47s
202:	learn: 0.4016860	total: 2m 27s	remaining: 1m 46s
203:	learn: 0.4011489	total: 2m 27s	remaining: 1m 45s
204:	learn: 0.4008355	total: 2m 28s	remaining: 1m 44s
205:	learn: 0.4006013	total: 2m 28s	remaining: 1m 44s
206:	learn: 0.4002927	total: 2m 29s	remaining: 1m 43s
207:	learn: 0.3998268	total: 2m 30s	remaining: 1m 42s
208:	learn: 0.3992957	total: 2m 31s	remaining: 1m 41s
209:	learn: 0.3988993	total: 2m 31s	remaining: 1m 41s
210:	learn: 0.3981433	total: 2m 32s	remaining: 1m 40s
211:	learn: 0.3979069	total: 2m 33s	remaining: 1m 39s
212:	learn: 0.3972508	total: 2m 33s	remaining: 1m 38s
213:	learn: 0.3968986	total: 2m 34s	remaining: 1m 38s
214:	learn: 0.3965711	total: 2m 35s	remaining: 1m 37s
215:	learn: 0.3960155	total: 2m 35s	remaining: 1m 36s
216:	learn: 0.3955190	total: 2m 36s	remaining: 1m 35s
217:	learn: 0.3952607	total:

4:	learn: 0.8233055	total: 3.5s	remaining: 4m 1s
5:	learn: 0.7913966	total: 4.32s	remaining: 4m 7s
6:	learn: 0.7644009	total: 5.25s	remaining: 4m 17s
7:	learn: 0.7412137	total: 5.99s	remaining: 4m 16s
8:	learn: 0.7213823	total: 6.76s	remaining: 4m 16s
9:	learn: 0.7028505	total: 7.49s	remaining: 4m 14s
10:	learn: 0.6865247	total: 8.26s	remaining: 4m 14s
11:	learn: 0.6722523	total: 8.9s	remaining: 4m 10s
12:	learn: 0.6601748	total: 9.69s	remaining: 4m 11s
13:	learn: 0.6482561	total: 10.3s	remaining: 4m 6s
14:	learn: 0.6382105	total: 10.9s	remaining: 4m 4s
15:	learn: 0.6291264	total: 11.8s	remaining: 4m 5s
16:	learn: 0.6209935	total: 12.6s	remaining: 4m 5s
17:	learn: 0.6133113	total: 13.4s	remaining: 4m 6s
18:	learn: 0.6064312	total: 14.1s	remaining: 4m 6s
19:	learn: 0.6004649	total: 15s	remaining: 4m 7s
20:	learn: 0.5942432	total: 15.8s	remaining: 4m 6s
21:	learn: 0.5890115	total: 16.5s	remaining: 4m 6s
22:	learn: 0.5838144	total: 17.3s	remaining: 4m 5s
23:	learn: 0.5799261	total: 18s	re

161:	learn: 0.4181410	total: 1m 55s	remaining: 2m 14s
162:	learn: 0.4179408	total: 1m 56s	remaining: 2m 13s
163:	learn: 0.4170805	total: 1m 57s	remaining: 2m 12s
164:	learn: 0.4165528	total: 1m 57s	remaining: 2m 12s
165:	learn: 0.4161485	total: 1m 58s	remaining: 2m 11s
166:	learn: 0.4155745	total: 1m 59s	remaining: 2m 10s
167:	learn: 0.4153871	total: 1m 59s	remaining: 2m 9s
168:	learn: 0.4147248	total: 2m	remaining: 2m 9s
169:	learn: 0.4144067	total: 2m 1s	remaining: 2m 8s
170:	learn: 0.4136360	total: 2m 1s	remaining: 2m 7s
171:	learn: 0.4128861	total: 2m 2s	remaining: 2m 6s
172:	learn: 0.4120392	total: 2m 3s	remaining: 2m 6s
173:	learn: 0.4112041	total: 2m 4s	remaining: 2m 5s
174:	learn: 0.4106575	total: 2m 4s	remaining: 2m 4s
175:	learn: 0.4099929	total: 2m 5s	remaining: 2m 4s
176:	learn: 0.4097875	total: 2m 6s	remaining: 2m 3s
177:	learn: 0.4096535	total: 2m 6s	remaining: 2m 2s
178:	learn: 0.4095364	total: 2m 7s	remaining: 2m 1s
179:	learn: 0.4087496	total: 2m 8s	remaining: 2m 1s
18

316:	learn: 0.3439025	total: 3m 42s	remaining: 23.2s
317:	learn: 0.3437385	total: 3m 43s	remaining: 22.5s
318:	learn: 0.3435483	total: 3m 44s	remaining: 21.8s
319:	learn: 0.3433256	total: 3m 45s	remaining: 21.1s
320:	learn: 0.3429196	total: 3m 45s	remaining: 20.4s
321:	learn: 0.3423035	total: 3m 46s	remaining: 19.7s
322:	learn: 0.3419304	total: 3m 47s	remaining: 19s
323:	learn: 0.3414904	total: 3m 47s	remaining: 18.3s
324:	learn: 0.3409841	total: 3m 48s	remaining: 17.6s
325:	learn: 0.3407344	total: 3m 49s	remaining: 16.9s
326:	learn: 0.3405021	total: 3m 49s	remaining: 16.2s
327:	learn: 0.3403454	total: 3m 50s	remaining: 15.5s
328:	learn: 0.3400734	total: 3m 51s	remaining: 14.8s
329:	learn: 0.3395799	total: 3m 51s	remaining: 14.1s
330:	learn: 0.3393424	total: 3m 52s	remaining: 13.3s
331:	learn: 0.3388804	total: 3m 53s	remaining: 12.6s
332:	learn: 0.3382884	total: 3m 53s	remaining: 11.9s
333:	learn: 0.3377651	total: 3m 54s	remaining: 11.2s
334:	learn: 0.3373628	total: 3m 55s	remaining: 1

In [ ]:
Cat_8127.fit(x_train_2,y_train_2)

0:	learn: 1.0187846	total: 733ms	remaining: 4m 15s
1:	learn: 0.9543063	total: 1.43s	remaining: 4m 8s
2:	learn: 0.9030945	total: 2.15s	remaining: 4m 8s
3:	learn: 0.8587966	total: 2.78s	remaining: 4m
4:	learn: 0.8213831	total: 3.43s	remaining: 3m 56s
5:	learn: 0.7898780	total: 4.07s	remaining: 3m 53s
6:	learn: 0.7640097	total: 4.77s	remaining: 3m 53s
7:	learn: 0.7392438	total: 5.43s	remaining: 3m 52s
8:	learn: 0.7193832	total: 6.22s	remaining: 3m 55s
9:	learn: 0.7014090	total: 6.93s	remaining: 3m 55s
10:	learn: 0.6853621	total: 7.63s	remaining: 3m 55s
11:	learn: 0.6704419	total: 8.39s	remaining: 3m 56s
12:	learn: 0.6576645	total: 8.94s	remaining: 3m 51s
13:	learn: 0.6455592	total: 9.64s	remaining: 3m 51s
14:	learn: 0.6335305	total: 10.3s	remaining: 3m 50s
15:	learn: 0.6228480	total: 11s	remaining: 3m 50s
16:	learn: 0.6141860	total: 11.7s	remaining: 3m 50s
17:	learn: 0.6055604	total: 12.4s	remaining: 3m 48s
18:	learn: 0.5986696	total: 13.1s	remaining: 3m 48s
19:	learn: 0.5928729	total: 13

157:	learn: 0.4207204	total: 2m 8s	remaining: 2m 35s
158:	learn: 0.4203140	total: 2m 8s	remaining: 2m 34s
159:	learn: 0.4199397	total: 2m 9s	remaining: 2m 34s
160:	learn: 0.4194309	total: 2m 10s	remaining: 2m 33s
161:	learn: 0.4191086	total: 2m 11s	remaining: 2m 32s
162:	learn: 0.4186509	total: 2m 12s	remaining: 2m 31s
163:	learn: 0.4181483	total: 2m 13s	remaining: 2m 31s
164:	learn: 0.4176675	total: 2m 14s	remaining: 2m 30s
165:	learn: 0.4171513	total: 2m 14s	remaining: 2m 29s
166:	learn: 0.4165883	total: 2m 15s	remaining: 2m 28s
167:	learn: 0.4160895	total: 2m 16s	remaining: 2m 27s
168:	learn: 0.4152286	total: 2m 17s	remaining: 2m 27s
169:	learn: 0.4149103	total: 2m 18s	remaining: 2m 26s
170:	learn: 0.4145020	total: 2m 18s	remaining: 2m 25s
171:	learn: 0.4138298	total: 2m 19s	remaining: 2m 24s
172:	learn: 0.4132827	total: 2m 20s	remaining: 2m 23s
173:	learn: 0.4129985	total: 2m 21s	remaining: 2m 22s
174:	learn: 0.4127012	total: 2m 21s	remaining: 2m 21s
175:	learn: 0.4124665	total: 2m

311:	learn: 0.3576160	total: 4m 11s	remaining: 30.7s
312:	learn: 0.3574247	total: 4m 12s	remaining: 29.9s
313:	learn: 0.3569995	total: 4m 13s	remaining: 29.1s
314:	learn: 0.3564049	total: 4m 14s	remaining: 28.3s
315:	learn: 0.3561456	total: 4m 15s	remaining: 27.5s
316:	learn: 0.3556214	total: 4m 16s	remaining: 26.7s
317:	learn: 0.3552622	total: 4m 17s	remaining: 25.9s
318:	learn: 0.3551319	total: 4m 18s	remaining: 25.1s
319:	learn: 0.3547121	total: 4m 19s	remaining: 24.3s
320:	learn: 0.3545022	total: 4m 19s	remaining: 23.5s
321:	learn: 0.3539631	total: 4m 21s	remaining: 22.7s
322:	learn: 0.3532479	total: 4m 21s	remaining: 21.9s
323:	learn: 0.3527455	total: 4m 22s	remaining: 21.1s
324:	learn: 0.3520673	total: 4m 23s	remaining: 20.3s
325:	learn: 0.3517277	total: 4m 24s	remaining: 19.5s
326:	learn: 0.3515797	total: 4m 25s	remaining: 18.6s
327:	learn: 0.3513902	total: 4m 25s	remaining: 17.8s
328:	learn: 0.3511066	total: 4m 26s	remaining: 17s
329:	learn: 0.3510131	total: 4m 27s	remaining: 1

In [ ]:
cv_scores

array([0.80723906, 0.8035113 , 0.80435305, 0.81072631, 0.79906205])

In [ ]:
np.mean(cv_scores)

0.804978354978355

In [ ]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,Cat_8127)


 Train accuracy 0.88

 Train Classification Report

                          precision    recall  f1-score   support

             functional       0.86      0.96      0.90     22535
functional needs repair       0.79      0.43      0.56      3024
         non functional       0.93      0.86      0.89     16021

               accuracy                           0.88     41580
              macro avg       0.86      0.75      0.79     41580
           weighted avg       0.88      0.88      0.87     41580


 Test accuracy 0.81

 Test Classification Report

                          precision    recall  f1-score   support

             functional       0.80      0.91      0.85      9724
functional needs repair       0.62      0.30      0.41      1293
         non functional       0.85      0.77      0.81      6803

               accuracy                           0.81     17820
              macro avg       0.76      0.66      0.69     17820
           weighted avg       0.81      0.81

In [ ]:
Cat_8127.get_feature_importance(prettified=True)

,Feature Id,Importances
0,quantity,9.992397
1,ward,9.046260
2,lga,7.647311
3,extraction_type,7.540403
4,funder,5.154053
5,waterpoint_type,4.713802
6,construction_year_cat,4.313144
7,source_type,3.857131
8,subvillage,3.852046
9,population,3.797049
